# Codigo EEMRIO by Martínez-Hernando


This information is necessary in Python, for activate the funtions 
If they are not installed is necessary to do it, write pip install XXXXX. If pip doesn't work put pip3, or pip3.11 (search on internet)

Activate generical information

In [1]:
import pandas as pd #To real CSV and dataframes
import numpy as np #To work with matrix
from scipy import io, integrate, linalg, signal #Google told me, I don´t know what is it
from scipy.sparse.linalg import cg, eigs #Google told me, I don´t know what is it

To create list

In [26]:
from typing import List, Tuple, Dict, Callable, Iterable, Union
import itertools
import json
from pathlib import Path

## Importing EXIOBASE 3.3.18 data
We will only import:
- $Z$, the intermediate consumption,
- $Y$, the final demand,
- $F$ and $F_Y$, the environmental extensions, only for air emissions.

In [3]:
exiobase_folder="P:/Universidad/Exiobase/" #Put here your folder where you have download Exiobase with /, no with \

Z (Intermediate products)

In [4]:
Z_raw=pd.read_csv(exiobase_folder + "MR_HIOT_2011_v3_3_18_by_product_technology.csv", #You should have this name of file, if not change it
                    index_col = [0,1,2,3,4],
                    header = [0,1,2,3])
Z_raw.index.names=["region","sector","code1","code2","units"]
Z_raw.head() #To see the first lines

AU  \
                                                   Cultivation of paddy rice   
                                                                       i01.a   
                                                                      A_PARI   
region sector                  code1 code2  units                              
AU     Paddy rice              p01.a C_PARI tonnes                       0.0   
       Wheat                   p01.b C_WHEA tonnes                       0.0   
       Cereal grains nec       p01.c C_OCER tonnes                       0.0   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                       0.0   
       Oil seeds               p01.e C_OILS tonnes                       0.0   

                                                                         \
                                                   Cultivation of wheat   
                                                                  i01.b   
                                                                 A_WHEA   
region sector                  code1 code2  units                         
AU     Paddy rice              p01.a C_PARI tonnes                  0.0   
       Wheat                   p01.b C_WHEA tonnes                  0.0   
       Cereal grains nec       p01.c C_OCER tonnes                  0.0   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                  0.0   
       Oil seeds               p01.e C_OILS tonnes                  0.0   

                                                                                     \
                                                   Cultivation of cereal grains nec   
                                                                              i01.c   
                                                                             A_OCER   
region sector                  code1 code2  units                                     
AU     Paddy rice              p01.a C_PARI tonnes                          0.00000   
       Wheat                   p01.b C_WHEA tonnes                          0.00000   
       Cereal grains nec       p01.c C_OCER tonnes                      70942.96547   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                          0.00000   
       Oil seeds               p01.e C_OILS tonnes                          0.00000   

                                                                                           \
                                                   Cultivation of vegetables, fruit, nuts   
                                                                                    i01.d   
                                                                                   A_FVEG   
region sector                  code1 code2  units                                           
AU     Paddy rice              p01.a C_PARI tonnes                                0.00000   
       Wheat                   p01.b C_WHEA tonnes                                0.00000   
       Cereal grains nec       p01.c C_OCER tonnes                                0.00000   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                            18266.75262   
       Oil seeds               p01.e C_OILS tonnes                                0.00000   

                                                                             \
                                                   Cultivation of oil seeds   
                                                                      i01.e   
                                                                     A_OILS   
region sector                  code1 code2  units                             
AU     Paddy rice              p01.a C_PARI tonnes                      0.0   
       Wheat                   p01.b C_WHEA tonnes                      0.0   
       Cereal grains nec       p01.c C_OCER tonnes                      0.0   
       Vegetables, fruit, nuts p01.d C_FVEG tonnes                      0.0   
       Oi

In [5]:
Z_reindexed=Z_raw.droplevel([2,3,4], axis=0).copy() #To remove useless rows
Z_reindexed.index.names=["region","sector"]
Z_reindexed.columns=Z_reindexed.index
Z_reindexed.head()


region                                 AU                          \
sector                         Paddy rice Wheat Cereal grains nec   
region sector                                                       
AU     Paddy rice                     0.0   0.0           0.00000   
       Wheat                          0.0   0.0           0.00000   
       Cereal grains nec              0.0   0.0       70942.96547   
       Vegetables, fruit, nuts        0.0   0.0           0.00000   
       Oil seeds                      0.0   0.0           0.00000   

region                                                            \
sector                         Vegetables, fruit, nuts Oil seeds   
region sector                                                      
AU     Paddy rice                              0.00000       0.0   
       Wheat                                   0.00000       0.0   
       Cereal grains nec                       0.00000       0.0   
       Vegetables, fruit, nuts             18266.75262       0.0   
       Oil seeds                               0.00000       0.0   

region                                                                    \
sector                         Sugar cane, sugar beet Plant-based fibers   
region sector                                                              
AU     Paddy rice                                 0.0               0.00   
       Wheat                                      0.0               0.00   
       Cereal grains nec                          0.0               0.00   
       Vegetables, fruit, nuts                    0.0               0.00   
       Oil seeds                                  0.0        -1206065.87   

region                                                                 ...  \
sector                         Crops nec        Cattle           Pigs  ...   
region sector                                                          ...   
AU     Paddy rice                    0.0  0.000000e+00       0.000000  ...   
       Wheat                         0.0  6.791574e+05  156083.893300  ...   
       Cereal grains nec             0.0  2.277000e+06  823314.442100  ...   
       Vegetables, fruit, nuts       0.0  1.552051e+03     255.322381  ...   
       Oil seeds                     0.0  0.000000e+00       0.000000  ...   

region                                                    WM  \
sector                         Paper for treatment: landfill   
region sector                                                  
AU     Paddy rice                                        0.0   
       Wheat                                             0.0   
       Cereal grains nec                                 0.0   
       Vegetables, fruit, nuts                           0.0   
       Oil seeds                                         0.0   

region                                                                \
sector                         Plastic waste for treatment: landfill   
region sector                                                          
AU     Paddy rice                                                0.0   
       Wheat                                                     0.0   
       Cereal grains nec                                         0.0   
       Vegetables, fruit, nuts                                   0.0   
       Oil seeds                                                 0.0   

region                                                                              \
sector                         Inert/metal/hazardous waste for treatment: landfill   
region sector                                                                        
AU     Paddy rice                                                             0.0    
       Wheat                                                                  0.0    
       Cereal grains nec                                                      0.0    
       Vegetables, fruit, nuts                                

In [6]:
exio3318_Z=Z_reindexed

Y (Final demand)

In [7]:
Y_raw = pd.read_csv(exiobase_folder + "MR_HIOT_2011_v3_3_18_FD.csv", #You should have this name of file, if not change it
                    index_col = [0,1,2,3,4],
                    header = [0,1,2,3])

Y_reindexed=Y_raw.droplevel([2,3], axis=1).copy() #To remove useless columns
Y_reindexed.index=Z_reindexed.index
Y_reindexed.columns.names=["region", "sector"] #To name the columns
exio3318_Y=Y_reindexed

In [8]:
exio3318_Y.head()

region                                                                  AU  \
sector                         Final consumption expenditure by households   
region sector                                                                
AU     Paddy rice                                             0.000000e+00   
       Wheat                                                  0.000000e+00   
       Cereal grains nec                                      0.000000e+00   
       Vegetables, fruit, nuts                                1.255628e+06   
       Oil seeds                                              7.501443e+02   

region                                                                                                               \
sector                         Final consumption expenditure by non-profit organisations serving households (NPISH)   
region sector                                                                                                         
AU     Paddy rice                                                        0.000000                                     
       Wheat                                                        317232.259200                                     
       Cereal grains nec                                             20796.255720                                     
       Vegetables, fruit, nuts                                        9066.727501                                     
       Oil seeds                                                       414.340678                                     

region                                                                      \
sector                         Final consumption expenditure by government   
region sector                                                                
AU     Paddy rice                                                 0.000000   
       Wheat                                                 635980.576100   
       Cereal grains nec                                      43596.865240   
       Vegetables, fruit, nuts                                13660.087220   
       Oil seeds                                                402.405072   

region                                                        \
sector                         Gross fixed capital formation   
region sector                                                  
AU     Paddy rice                                        0.0   
       Wheat                                             0.0   
       Cereal grains nec                                 0.0   
       Vegetables, fruit, nuts                           0.0   
       Oil seeds                                         0.0   

region                                                                      \
sector                         Changes in inventories Changes in valuables   
region sector                                                                
AU     Paddy rice                             0.00000                    0   
       Wheat                                  0.00000                    0   
       Cereal grains nec                      0.00000                    0   
       Vegetables, fruit, nuts            25217.20699                    0   
       Oil seeds                          11344.59956                    0   

region                                                                  AT  \
sector                         Final consumption expenditure by households   
region sector                                                                
AU     Paddy rice                                                 0.000000   
       Wheat                                                      0.000000   
       Cereal grains nec                                          0.645819   
       Vegetables, fruit, nuts                                  262.282173   
       Oil seeds                                                  0.000000   

region                            

F and F_Y (Extensions, total emissions per activity)

In [9]:
F_raw = pd.read_excel(exiobase_folder + "MR_HIOT_2011_v3_3_18_extensions.xlsx", #This data is emissions of intermediate production
                      sheet_name="Emiss_act", #You should have this name of file, if not change it.
                      index_col=[0,1,2],
                      header=[0,1,2,3])
F_raw.columns=Z_reindexed.columns

F_Y_raw=pd.read_excel(exiobase_folder + "MR_HIOT_2011_v3_3_18_extensions.xlsx", #This data is emissions of final demand
                      sheet_name="Emiss_FD", #You should have this name of file, if not change it.
                      index_col=[0,1,2],
                      header=[0,1,2,3])
F_Y_raw.columns=Y_reindexed.columns

F_raw.head()
F_Y_raw.head()
exio3318_emissions_F=F_raw
exio3318_emissions_F_Y=F_Y_raw

In [10]:
F_Y_raw.head()

region                                                                     AU  \
sector                            Final consumption expenditure by households   
Carbon dioxide, fossil tonnes air                                4.760370e+07   
N2O                    tonnes air                                9.248053e+02   
CH4                    tonnes air                                4.907784e+04   
HFCs                   tonnes air                                0.000000e+00   
PFCs                   tonnes air                                0.000000e+00   

region                                                                                                                  \
sector                            Final consumption expenditure by non-profit organisations serving households (NPISH)   
Carbon dioxide, fossil tonnes air                                           0.000000                                     
N2O                    tonnes air                                           0.000000                                     
CH4                    tonnes air                                         200.749203                                     
HFCs                   tonnes air                                           0.000000                                     
PFCs                   tonnes air                                           0.000000                                     

region                                                                         \
sector                            Final consumption expenditure by government   
Carbon dioxide, fossil tonnes air                                    0.000000   
N2O                    tonnes air                                    0.000000   
CH4                    tonnes air                                  343.422972   
HFCs                   tonnes air                                    0.000000   
PFCs                   tonnes air                                    0.000000   

region                                                           \
sector                            Gross fixed capital formation   
Carbon dioxide, fossil tonnes air                             0   
N2O                    tonnes air                             0   
CH4                    tonnes air                             0   
HFCs                   tonnes air                             0   
PFCs                   tonnes air                             0   

region                                                                         \
sector                            Changes in inventories Changes in valuables   
Carbon dioxide, fossil tonnes air                      0                    0   
N2O                    tonnes air                      0                    0   
CH4                    tonnes air                      0                    0   
HFCs                   tonnes air                      0                    0   
PFCs                   tonnes air                      0                    0   

region                                                                     AT  \
sector                            Final consumption expenditure by households   
Carbon dioxide, fossil tonnes air                                2.287570e+07   
N2O                    tonnes air                                6.054108e+02   
CH4                    tonnes air                                1.750280e+04   
HFCs                   tonnes air                                0.000000e+00   
PFCs                   tonnes air                                0.000000e+00   

region                                                                                                                  \
sector                            Final consumption expenditure by non-profit organisations serving households (NPISH)   
Carbon dioxide, fossil tonnes air                                                  0                                     
N2O                    tonnes air    

In [11]:
X_raw = pd.read_csv(exiobase_folder + "MR_HIOT_2011_v3_3_18_principal_production.csv", #You should have this name of file, if not change it
                    header = [0,1,2,3,4,5,6,7])

In [13]:
X_reindexed=X_raw.droplevel([2,3,4,5,6,7], axis=1).copy()
X_reindexed.columns=exio3318_Z.columns #To rename the columns
X_reindexed.index=['Total production'] #To rename the index
exio3318_X=X_reindexed

In [14]:
A=exio3318_Z/exio3318_X.squeeze() #To calculate the matrix A
A=A.fillna(0) #To fill the NaN values with 0
n= len(A.index) #To know the number of rows
Ident=np.eye(n) #To create the identity matrix
I_A=Ident-A #To calculate the matrix I-A

In [15]:
print(I_A.shape) #To know the shape of the matrix
I_A.isna().any().any()
print(type(I_A)) #To know the type of the matrix

(7872, 7872)
<class 'pandas.core.frame.DataFrame'>


In [16]:
exio3318_emissions_S=exio3318_emissions_F/exio3318_X.squeeze()
exio3318_emissions_S=exio3318_emissions_S.fillna(0)

# Importation for BW2.5

Libraries for brightway2.5

In [1]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
from bw2io.importers.io import IOImporter
from bw2io.strategies.io import tidy_tables

print('bw2data version: ',bd.__version__)
print('bw2io version: ',bi.__version__)
print('bw2calc version: ',bc.__version__)

bd.projects
bd.projects.set_current('exiobase_db')

if any("forwast" in db for db in bd.databases): #to confirm biosphere is present
    print('forwast is already present in the project.')
else:
    bi.remote.get_projects()
    bi.remote.install_project('forwast','exiobase_db', overwrite_existing=True)

17:59:13+0200 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.
bw2data version:  (4, 0)
bw2io version:  0.9.3
bw2calc version:  2.0.2
forwast is already present in the project.


To confirm biosphere is well installed

In [20]:
methods_installed=len(bd.methods) #This number is not 1, if it is 1; try to run in an other kernel
print(bd.databases)
if methods_installed==1:
    print('You have a problem in brightway installation, only one mtehod have been added. It will be deleted and intalled again')

Databases dictionary with 3 object(s):
	Emissions
	ecoinvent-3.9.1-biosphere
	forwast


# Creation of matrix needed

Index needed

In [21]:
countries=exio3318_Z.columns.get_level_values(0)
products=exio3318_Z.columns.get_level_values(1)
nick_names=[f"{coun}_{prod}" for coun, prod in zip(countries, products)]
units=np.array(Z_raw.index.get_level_values("units")) #To see the units

sustances=exio3318_emissions_S.index.get_level_values(0)
unit_emissions=exio3318_emissions_S.index.get_level_values(1)
compartments=exio3318_emissions_S.index.get_level_values(2)


In [22]:
compartments[0]

'air'

# Create database

In [27]:
path_to_intermediate=(Path.cwd()/'example_data')
path_to_intermediate.mkdir(exist_ok=True)

In [28]:
extended_hiot=I_A
extended_S=exio3318_emissions_S

extended_S=extended_S.droplevel([1,2], axis=0).copy() #To name S as in biosphere3
extended_S=extended_S[0:3]
extended_S.rename(index={'Carbon dioxide, fossil': 'co2_air','N2O':'n2o_air','CH4':'ch4_air'}, inplace=True)

for col in extended_hiot.columns:
    extended_hiot[col] = extended_hiot[col].astype(pd.SparseDtype("float", 0))
for col in extended_S.columns:
    extended_S[col] = extended_S[col].astype(pd.SparseDtype("float", 0))


In [29]:
# Para extended_hiot
print(f"extended_hiot es sparse: {all(dtype == 'Sparse[float, 0]' for dtype in extended_hiot.dtypes)}")

# Para extended_S
print(f"extended_S es sparse: {all(dtype == 'Sparse[float, 0]' for dtype in extended_S.dtypes)}")

extended_hiot es sparse: False
extended_S es sparse: False


In [30]:
print(extended_S)

region          AU                                                      \
sector  Paddy rice     Wheat Cereal grains nec Vegetables, fruit, nuts   
co2_air   0.061182  0.083341          0.048647                0.075276   
n2o_air   0.000072  0.000407          0.000385                0.002579   
ch4_air   0.043148  0.000007          0.000014                0.000021   

region                                                                 \
sector  Oil seeds Sugar cane, sugar beet Plant-based fibers Crops nec   
co2_air  0.078753                0.02291           0.195253  2.428213   
n2o_air  0.000439               0.000154           0.001501  0.001398   
ch4_air  0.000019               0.000003           0.000016   0.00025   

region                       ...                            WM  \
sector     Cattle      Pigs  ... Paper for treatment: landfill   
co2_air  2.517309  0.052053  ...                      0.000015   
n2o_air  0.066276  0.001816  ...                             0   

In [42]:
product_metadata={
    name: {
        "unit": unit,
        "name": name,
        "location": region,
    }
    for name, unit, region in zip(products, units, countries)
}
biosphere_metadata = {
    'co2_air':{'name':sustances[0],
               'unit':unit_emissions[0], # not standard units
               'compartment':compartments[0],
               },
    'n2o_air':{'name':sustances[1],
               'unit':unit_emissions[1], # not standard units
               'compartment':compartments[1]},
    'ch4_air':{'name':sustances[2],
               'unit':unit_emissions[2], # not standard units
               'compartment':compartments[2]},               
}
metadata_dict = {**product_metadata, **biosphere_metadata}

with open(path_to_intermediate/'io_metadata.json', 'w') as fp:
    json.dump(metadata_dict, fp,indent=4)


In [43]:
co2_flows = [act for act in bd.Database('ecoinvent-3.9.1-biosphere') if 'carbon dioxide, fossil' in act['name'].lower()] #To select the key for emissions
n2o_flows = [act for act in bd.Database('ecoinvent-3.9.1-biosphere') if 'dinitrogen monoxide' in act['name'].lower()]
ch4_flows = [act for act in bd.Database('ecoinvent-3.9.1-biosphere') if 'methane, fossil' in act['name'].lower()]

biosphere_mapping = {
    'co2_air':co2_flows[0]['code'],
    'n2o_air':n2o_flows[0]['code'],
    'ch4_air':ch4_flows[0]['code'],
}

In [44]:
biosphere_mapping

{'co2_air': 'e259263c-d1f1-449f-bb9b-73c6d0a32a00',
 'n2o_air': '6dc1b46f-ee89-4495-95c4-b8a637bcd6cb',
 'ch4_air': '70ef743b-3ed5-4a6d-b192-fb6d62378555'}

In [45]:
print("Columns:", extended_hiot.columns.nlevels) #It must be 2
print("Index:", extended_hiot.index.nlevels) #It must be 2

Columns: 2
Index: 2


In [46]:
tidy_tables(extended_hiot,extended_S,path_to_intermediate)

In [47]:
name_database="exiobase_318_hybrid" 
if name_database in bd.databases:
    del bd.databases[name_database]

In [49]:
exiobase_try1_bw= IOImporter(dirpath=path_to_intermediate,
                         db_name='exiobase_318_hybrid',
                         b3mapping=biosphere_mapping)
exiobase_try1_bw.apply_strategies()
exiobase_try1_bw.write_database()

no extra biosphere flows added


100%|██████████| 7872/7872 [00:00<00:00, 11156.94it/s]


17:29:25+0200 [info     ] Vacuuming database            
17:29:26+0200 [info     ] Starting IO table write       
17:29:27+0200 [info     ] Adding technosphere matrix    
17:31:21+0200 [info     ] Adding biosphere matrix       
17:31:21+0200 [info     ] Finalizing serialization      


In [2]:
test= bd.Database('exiobase_318_hybrid')

In [3]:
locations = {a['location'] for a in test}
names = {a['name'] for a in test}

In [4]:
print(f"{len(locations)} locations, {len(names)} different products")

48 locations, 164 different products


In [5]:
ipcc_2021 = ('IPCC 2021', 'climate change', 'global warming potential (GWP100)')

In [6]:
act=test.random()
act
print(type(act))
act2=next((a for a in test if a['name'] == 'Paddy rice' and a['location'] == 'AU'), None)
print(act2)
print(type(act2))

<class 'bw2data.backends.iotable.proxies.IOTableActivity'>
'Paddy rice' (ton, AU, None)
<class 'bw2data.backends.iotable.proxies.IOTableActivity'>


In [7]:
lca = bc.LCA(
    demand={act2: 1},
    method=ipcc_2021 
)
lca.lci()
lca.lcia()
lca.score
print(lca.score)



1755.1084916676612


In [8]:
lca.to_dataframe().pivot_table(index='row_name',aggfunc='sum',values='amount')

,amount
row_name,
"Carbon dioxide, fossil",494.873968
Dinitrogen monoxide,26.977239
"Methane, fossil",1225.860615
